In [ ]:
import pandas as pd
# sys.path.insert(0, 'D:\GitHub Repos\MLib in Python\MLib-in-python')
# import MLib
import warnings
warnings.filterwarnings("ignore")


In [ ]:
DFOriginal = pd.read_csv('csv-wsn-ids-ds.csv')
OutPutLabel : str = 'Attack type'
InputLabels : list = DFOriginal.columns.tolist()
InputLabels.remove(OutPutLabel)
print('Input Labels Count: ', len(InputLabels))
InputLabels

In [ ]:
DFOriginal[OutPutLabel].value_counts().index.tolist()

In [ ]:
DFOriginal.dtypes

In [ ]:
DFOriginal.info()

In [ ]:
# Display the number of missing values in each column
DFOriginal.isnull().sum()

In [ ]:
DFOriginal.describe()

In [ ]:
# Display the number of unique values in each column
DFOriginal.nunique()

In [ ]:
DFOriginal.drop_duplicates(inplace=True)

In [ ]:
DFOriginal.info()

In [ ]:
DFOriginal.hist(figsize=(20, 15), bins=50, xlabelsize=8, ylabelsize=8)

In [ ]:
DFOriginal[OutPutLabel].value_counts()

In [ ]:
# Display the first few rows of the dataset
DFOriginal.head()

In [ ]:
DFOriginalInputs = DFOriginal[InputLabels]
DFOriginalOutput = DFOriginal[OutPutLabel]

In [ ]:
# print th correlation matrix with colored cells
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 8))
sns.heatmap(DFOriginalInputs.corr(), annot=True, fmt=".1f", cmap='coolwarm', square=True, cbar_kws={"shrink": .8})
plt.title('Correlation Matrix')
plt.show()


In [ ]:
DFOriginalOutput.value_counts().plot(kind='bar')
plt.title('Class Distribution of Attack Types')
plt.xlabel('Attack Type')
plt.ylabel('Frequency')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(DFInputs, DFOutput, test_size=0.2, stratify=DFOutput)
from imblearn.over_sampling import SMOTE
smote = SMOTE()
DFSampledInputs, DFSampledOutput = smote.fit_resample(DFOriginalInputs, DFOriginalOutput)
# merge thsamplede  inputs and outputs back into a single DataFrame
DFSampled = pd.concat([DFSampledInputs, DFSampledOutput], axis=1)

In [ ]:
DFSampledOutput.value_counts().plot(kind='bar')
plt.title('Class Distribution of Attack Types')
plt.xlabel('Attack Type')
plt.ylabel('Frequency')
plt.xticks(rotation=45)
plt.show()

In [ ]:
DFSampledOutput.value_counts()  

In [ ]:
# use LabelEncoder to convert the categorical target to numerical values
def LabelEncoding(DF : pd.DataFrame) -> pd.DataFrame:
    from sklearn.preprocessing import LabelEncoder
    categorical_columns = OutPutLabel
    le = LabelEncoder()
    encoded = le.fit_transform(DF[categorical_columns])
    encoded_df = pd.DataFrame(encoded, columns=[categorical_columns])
    DFLE = pd.concat([DF.drop(columns=[categorical_columns]), encoded_df], axis=1) # there is a problem here, let's fix it
    return DFLE

In [ ]:
def OneHotEncoding(DF : pd.DataFrame) -> pd.DataFrame:
    from sklearn.preprocessing import OneHotEncoder
    categorical_columns = DF.select_dtypes(include=['object']).columns.tolist()
    ohe = OneHotEncoder(sparse_output=False)
    encoded = ohe.fit_transform(DF[categorical_columns])
    encoded_df = pd.DataFrame(encoded, columns=ohe.get_feature_names_out(categorical_columns))
    DFONE = pd.concat([DF.drop(columns=categorical_columns), encoded_df], axis=1)
    return DFONE


In [ ]:
le = LabelEncoding(DFSampled)
le.head()

In [ ]:
ohe = OneHotEncoding(DFSampled)
ohe.head()

In [ ]:
from copy import deepcopy
DF : pd.DataFrame = deepcopy(DFSampled)
DF = DFSampled.copy()
DFInputs = deepcopy(DFSampledInputs)
DFOutput = deepcopy(DFSampledOutput)

In [ ]:
# use MinMaxScaler to scale the data
def MinMaxScaling(DF : pd.DataFrame) -> pd.DataFrame:
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    DFarr = scaler.fit_transform(DF)
    DFMMS = pd.DataFrame(DFarr, columns=scaler.get_feature_names_out(DF.columns.tolist()))
    return DFMMS

In [ ]:
# use StandardScaler to scale the data
def StandardScaling(DF : pd.DataFrame) -> pd.DataFrame:
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    DFarr = scaler.fit_transform(DF)
    DFSS = pd.DataFrame(DFarr, columns=scaler.get_feature_names_out(DF.columns.tolist()))
    return DFSS

In [ ]:
# use RobustScaler to scale the data
def RobustScaling(DF : pd.DataFrame) -> pd.DataFrame:
    from sklearn.preprocessing import RobustScaler
    scaler = RobustScaler()
    DFarr = scaler.fit_transform(DF)
    DFRS = pd.DataFrame(DFarr, columns=scaler.get_feature_names_out(DF.columns.tolist()))
    return DFRS

In [ ]:
mms = MinMaxScaling(DFInputs)
mms.head()

In [ ]:
ss = StandardScaling(DFInputs)
ss.head()

In [ ]:
rs = RobustScaling(DFInputs)
rs.head()

In [ ]:
# use PCA to reduce the dimensionality of the data
def PCA(DF : pd.DataFrame, n_components: int) -> pd.DataFrame:
    from sklearn.decomposition import PCA
    pca = PCA(n_components=n_components)
    DFarr = pca.fit_transform(DF)
    DFpca = pd.DataFrame(DFarr, columns=[f'PC{i+1}' for i in range(n_components)])
    return DFpca
# use TSNE to reduce the dimensionality of the data
def TSNE(DF : pd.DataFrame, n_components: int) -> pd.DataFrame:
    from sklearn.manifold import TSNE
    tsne = TSNE(n_components=n_components)
    DFarr = tsne.fit_transform(DF)
    DFtsne = pd.DataFrame(DFarr, columns=[f'TSNE{i+1}' for i in range(n_components)])
    return DFtsne
# use UMAP to reduce the dimensionality of the data
def UMAP(DF : pd.DataFrame, n_components: int) -> pd.DataFrame:
    from umap import UMAP
    umap = UMAP(n_components=n_components)
    DFarr = umap.fit_transform(DF)
    DFumap = pd.DataFrame(DFarr, columns=[f'UMAP{i+1}' for i in range(n_components)])
    return DFumap
# use LDA to reduce the dimensionality of the data
def LDA(DF : pd.DataFrame, n_components: int) -> pd.DataFrame:
    from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
    lda = LinearDiscriminantAnalysis(n_components=n_components)
    DFarr = lda.fit_transform(DF, DFOutput)
    DFlda = pd.DataFrame(DFarr, columns=[f'LDA{i+1}' for i in range(n_components)])
    return DFlda
# use ICA to reduce the dimensionality of the data
def ICA(DF : pd.DataFrame, n_components: int) -> pd.DataFrame:
    from sklearn.decomposition import FastICA
    ica = FastICA(n_components=n_components)
    DFarr = ica.fit_transform(DF)
    DFica = pd.DataFrame(DFarr, columns=[f'ICA{i+1}' for i in range(n_components)])
    return DFica
# use NMF to reduce the dimensionality of the data
def NMF(DF : pd.DataFrame, n_components: int) -> pd.DataFrame:
    from sklearn.decomposition import NMF
    nmf = NMF(n_components=n_components)
    DFarr = nmf.fit_transform(DF)
    DFnmf = pd.DataFrame(DFarr, columns=[f'NMF{i+1}' for i in range(n_components)])
    return DFnmf
# use SVD to reduce the dimensionality of the data
def SVD(DF : pd.DataFrame, n_components: int) -> pd.DataFrame:
    from sklearn.decomposition import TruncatedSVD
    svd = TruncatedSVD(n_components=n_components)
    DFarr = svd.fit_transform(DF)
    DFsvd = pd.DataFrame(DFarr, columns=[f'SVD{i+1}' for i in range(n_components)])
    return DFsvd
# use FA to reduce the dimensionality of the data
def FA(DF : pd.DataFrame, n_components: int) -> pd.DataFrame:
    from sklearn.decomposition import FactorAnalysis
    fa = FactorAnalysis(n_components=n_components)
    DFarr = fa.fit_transform(DF)
    DFfa = pd.DataFrame(DFarr, columns=[f'FA{i+1}' for i in range(n_components)])
    return DFfa
# use KMeans to reduce the dimensionality of the data
def KMeans(DF : pd.DataFrame, n_components: int) -> pd.DataFrame:
    from sklearn.cluster import KMeans
    kmeans = KMeans(n_clusters=n_components)
    DFarr = kmeans.fit_transform(DF)
    DFkmeans = pd.DataFrame(DFarr, columns=[f'KMeans{i+1}' for i in range(n_components)])
    return DFkmeans
# use DBSCAN to reduce the dimensionality of the data
def DBSCAN(DF : pd.DataFrame, n_components: int) -> pd.DataFrame:
    from sklearn.cluster import DBSCAN
    dbscan = DBSCAN(eps=0.5, min_samples=5)
    DFarr = dbscan.fit_transform(DF)
    DFdbscan = pd.DataFrame(DFarr, columns=[f'DBSCAN{i+1}' for i in range(n_components)])
    return DFdbscan
# use AgglomerativeClustering to reduce the dimensionality of the data
def AgglomerativeClustering(DF : pd.DataFrame, n_components: int) -> pd.DataFrame:
    from sklearn.cluster import AgglomerativeClustering
    agglomerative = AgglomerativeClustering(n_clusters=n_components)
    DFarr = agglomerative.fit_transform(DF)
    DFagglomerative = pd.DataFrame(DFarr, columns=[f'Agglomerative{i+1}' for i in range(n_components)])
    return DFagglomerative
# use Birch to reduce the dimensionality of the data
def Birch(DF : pd.DataFrame, n_components: int) -> pd.DataFrame:
    from sklearn.cluster import Birch
    birch = Birch(n_clusters=n_components)
    DFarr = birch.fit_transform(DF)
    DFbirch = pd.DataFrame(DFarr, columns=[f'Birch{i+1}' for i in range(n_components)])
    return DFbirch
# use MeanShift to reduce the dimensionality of the data
def MeanShift(DF : pd.DataFrame, n_components: int) -> pd.DataFrame:
    from sklearn.cluster import MeanShift
    meanshift = MeanShift()
    DFarr = meanshift.fit_transform(DF)
    DFmeanshift = pd.DataFrame(DFarr, columns=[f'MeanShift{i+1}' for i in range(n_components)])
    return DFmeanshift
# use SpectralClustering to reduce the dimensionality of the data
def SpectralClustering(DF : pd.DataFrame, n_components: int) -> pd.DataFrame:
    from sklearn.cluster import SpectralClustering
    spectral = SpectralClustering(n_clusters=n_components)
    DFarr = spectral.fit_transform(DF)
    DFspectral = pd.DataFrame(DFarr, columns=[f'Spectral{i+1}' for i in range(n_components)])
    return DFspectral
# use OPTICS to reduce the dimensionality of the data
def OPTICS(DF : pd.DataFrame, n_components: int) -> pd.DataFrame:
    from sklearn.cluster import OPTICS
    optics = OPTICS()
    DFarr = optics.fit_transform(DF)
    DFoptics = pd.DataFrame(DFarr, columns=[f'OPTICS{i+1}' for i in range(n_components)])
    return DFoptics
# use AffinityPropagation to reduce the dimensionality of the data
def AffinityPropagation(DF : pd.DataFrame, n_components: int) -> pd.DataFrame:
    from sklearn.cluster import AffinityPropagation
    affinity = AffinityPropagation()
    DFarr = affinity.fit_transform(DF)
    DFaAffinity = pd.DataFrame(DFarr, columns=[f'Affinity{i+1}' for i in range(n_components)])
    return DFaAffinity
# use GaussianMixture to reduce the dimensionality of the data
def GaussianMixture(DF : pd.DataFrame, n_components: int) -> pd.DataFrame:
    from sklearn.mixture import GaussianMixture
    gmm = GaussianMixture(n_components=n_components)
    DFarr = gmm.fit_transform(DF)
    DFgmm = pd.DataFrame(DFarr, columns=[f'GaussianMixture{i+1}' for i in range(n_components)])
    return DFgmm

In [ ]:
DRFunctions = [PCA, TSNE, UMAP, LDA, ICA, NMF,
               SVD, FA, KMeans, DBSCAN, AgglomerativeClustering,
               Birch, MeanShift, SpectralClustering, OPTICS,
               AffinityPropagation, GaussianMixture]
DRFunctions.__len__()

In [ ]:
# HEREHEREHEREHEREHEREHEREHEREHEREHEREHEREHEREHEREHEREHEREHEREHERE

In [ ]:
"""
TODO: 
    - done: solve the impbalancing in the dataset
    - scale the dataset using:
        - MinMaxScaler 
        - StandardScaler 
        - RobustScaler
    - categorical encoding using:
        - LabelEncoder: for the OutPutLabel
        - OneHotEncoder: for the OutPutLabel
    - use dimensionality reduction techniques:
        - the techniques listed in the DRFunctions list

    - split the dataset into train and test sets
    - layout the different calssifiers we may use
        - NOTE: make a grid/random search for the best parameters for each classifier independently finish one by one
        - SGDClassifier
        - DecisionTreeClassifier
        - RandomForestClassifier
        - KNeighborsClassifier
        - MLPClassifier
        - SVC
        - SVM
        - LogisticRegression
        - AdaBoostClassifier
        - GradientBoostingClassifier
        - XGBClassifier
        - LGBMClassifier
        - ExtraTreesClassifier
        - BaggingClassifier
        - VotingClassifier
"""